In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
 cd /content/drive/Shareddrives/와썹_공개SW/GPT2_MNews/

/content/drive/Shareddrives/와썹_공개SW/GPT2_MNews


In [3]:
!pip install -r requirements.txt

     |████████████████████████████████| 256kB 12.8MB/s 
     |████████████████████████████████| 1.1MB 19.9MB/s 
     |████████████████████████████████| 675kB 53.1MB/s 
     |████████████████████████████████| 68.7MB 45kB/s 
     |████████████████████████████████| 890kB 48.7MB/s 
     |████████████████████████████████| 3.8MB 52.5MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.9.1-cp36-cp36m-linux_x86_64.whl size=470033 sha256=ce040569b86517ce74f651828fa0feb7e9707a3f10b8594f8cd9fb41a8158e46
  Stored in directory: /root/.cache/pip/wheels/af/60/16/1f8a40e68b85bd9bd7960e91830bca5e40cd113f3220b7e231
  Created wheel for kss: filename=kss-1.3.1-cp36-cp36m-linux_x86_64.whl size=251531 sha256=20746aa0eb00f1f69d11cabbf8bc93d87e5d366588c54a93717700ef8ef99db6
  Stored in directory: /root/.cache/pip/wheels/8b/98/d1/53f75f89925cd95779824778725ee3fa36e7aa55ed26ad54a8
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=1bc9a180a790af3ac0902a0702a0ea902

In [4]:
import os

import numpy as np
import tensorflow as tf

import gluonnlp as nlp
from gluonnlp.data import SentencepieceTokenizer
from transformers import TFGPT2LMHeadModel

from tensorflow.keras.preprocessing.sequence import pad_sequences

from nltk.tokenize import sent_tokenize

import pandas as pd
import kss
import random

from utils.utils import get_jongsung_TF
from tqdm import tqdm

In [5]:
class GPT2Model(tf.keras.Model):
    def __init__(self, dir_path):
        super(GPT2Model, self).__init__()
        self.gpt2 = TFGPT2LMHeadModel.from_pretrained(dir_path)
        
    def call(self, inputs):
        return self.gpt2(inputs)[0]

In [6]:
TOKENIZER_PATH = './gpt_ckpt/gpt2_kor_tokenizer.spiece'
tokenizer = SentencepieceTokenizer(TOKENIZER_PATH, num_best=0, alpha=0)
vocab = nlp.vocab.BERTVocab.from_sentencepiece(TOKENIZER_PATH,
                                               mask_token=None,
                                               sep_token=None,
                                               cls_token=None,
                                               unknown_token='<unk>',
                                               padding_token='<pad>',
                                               bos_token='<s>',
                                               eos_token='</s>')

## 텍스트 생성-코드

In [7]:
def tf_top_k_top_p_filtering(logits, top_k=0, top_p=0.0, filter_value=-99999):
    _logits = logits.numpy()
    top_k = min(top_k, logits.shape[-1])  
    if top_k > 0:
        indices_to_remove = logits < tf.math.top_k(logits, top_k)[0][..., -1, None]
        _logits[indices_to_remove] = filter_value

    if top_p > 0.0:
        sorted_logits = tf.sort(logits, direction='DESCENDING')
        sorted_indices = tf.argsort(logits, direction='DESCENDING')
        cumulative_probs = tf.math.cumsum(tf.nn.softmax(sorted_logits, axis=-1), axis=-1)

        sorted_indices_to_remove = cumulative_probs > top_p
        sorted_indices_to_remove = tf.concat([[False], sorted_indices_to_remove[..., :-1]], axis=0)
        indices_to_remove = sorted_indices[sorted_indices_to_remove].numpy().tolist()
        
        _logits[indices_to_remove] = filter_value
    return tf.constant([_logits])

In [8]:
# GPT Model을 활용한 텍스트 생성
def generate_sent(seed_word, model, max_step=400, greedy=False, top_k=0, top_p=0.):
    sent = seed_word
    toked = tokenizer(sent)
    for _ in tqdm(range(max_step)):
        input_ids = tf.constant([vocab[vocab.bos_token],]  + vocab[toked])[None, :] 
        outputs = model(input_ids)[:, -1, :]
        if greedy:
            gen = vocab.to_tokens(tf.argmax(outputs, axis=-1).numpy().tolist()[0])
        else:
            output_logit = tf_top_k_top_p_filtering(outputs[0], top_k=top_k, top_p=top_p)
            gen = vocab.to_tokens(tf.random.categorical(output_logit, 1).numpy().tolist()[0])[0]
        if gen == '</s>':
            break
        #print("생성된 토큰 : "+gen)
        sent += gen.replace('▁', ' ')
        toked = tokenizer(sent)
    return sent

In [9]:
# 불완전한 종결 문장 제거
def preprocessing(content):
    text = []
    article = ""
    for s in kss.split_sentences(content):
        text.append(s)
    if text[-1][-1] != '.':
        text = text[:-1]
    for t in text:
        article += t + " "
    return article

In [10]:
# DB에서 columns에 있는 열 중 랜덤한 정보를 가져온다.
def random_info(df, keyword, columns):
    column = random.choice(columns)
    for i in range(len(df)):
        if df.loc[i, 'disease'] == keyword:
            csv_text = df.loc[i, column]
            return csv_text 
    return ""

In [11]:
# 기사 생성
# return : title, content
def generate_article(keyword, title_path, content_path, df):
    title_model = GPT2Model(title_path)    # 제목 생성 모델 load
    title = generate_sent(keyword, title_model,top_k=0, top_p=0.90)     # 제목 생성 모델을 활용하여 기사 제목 생성 
    content_model = GPT2Model(content_path)     # 본문 생성 모델

    # 본문 생성 모델의 input 값으로 제목을 넣기 위한 전처리
    # 제목이 '.' 으로 끝나지 않을시 .을 추가한다.
    title = title.strip()
    if title[-1] not in ['.', '?', '!']:
        title += '.'
    # 사용할지 말지 임의로 정하기
    #if '...' in title:
    #    title = title.split('...')[-1]
    #    if title[-1] not in ['.', '?', '!']:
    #        title += '.'

    # 본문 생성모델의 input
    # 예) [기사 제목]. + [질병명] + 이란(란)
    input_text = title + keyword

    # 종성에 따라 조사 추가    
    if get_jongsung_TF(keyword) == "T":
        input_text += '이란'
    else:
        input_text += '란'

    content = generate_sent(input_text, content_model, top_k=40, top_p=0.90)    # 본문 생성 모델을 활용하여 기사 본문 생성
    content = content.replace(title, "").strip()
    content = preprocessing(content)        # 불완전한 종결문장 제거

    csv_text = ""   # DB로 부터 불러올 데이터
    columns = ['cause', 'symptom', 'diagnosis', 'treat']

    # 기사 본문에 질병의 증상, 진단, 치료, 원인을 다루는지 확인하고 없는 것만 columns에 추가
    if '증상' in content:
        columns.remove('symptom')
    if '진단' in content:
        columns.remove('diagnosis')
    if '치료' in content:
        columns.remove('treat')
    if '원인' in content:
        columns.remove('cause')

    if columns:     # columns의 원소들 중 하나를 랜덤하게 선택하여 질병 정보를 출력
        csv_text = random_info(df=df, keyword=keyword, columns = columns)

    content = content + csv_text    
    content = preprocessing(content)    # 불완전한 종결문장 제거, 공백 교정
    return title, content

# Generate Article

In [12]:
TITLE_MODEL_PATH = './mnews_title/maxlen20_loss0.36_acc0.56/'
CONTENT_MODEL_PATH = './final_model/psy_loss2.0_acc0.32/'

In [13]:
df = pd.read_csv("./data/samsung_hospital_db.csv")      # 질병 DB load

# textming 모델에서 LDA 분석을 통하여 추출된 keyword LOAD
input_keyword_df = pd.read_csv("./data/disease_keyword.csv")

# 추출된 keywords에서 랜덤하게 keyword(질병명) 선택 기사 선택
idx = random.randint(0, len(input_keyword_df)-1)    
keyword = input_keyword_df.loc[idx, 'disease']

print("Keyword:", keyword)

Keyword: 폐경


In [14]:
title, content = generate_article(keyword=keyword, title_path=TITLE_MODEL_PATH, content_path=CONTENT_MODEL_PATH, df=df)

100%|██████████| 400/400 [00:43<00:00,  9.23it/s]


# Article

In [15]:
title

'폐경무렵 생리량 늘면 건강하다? 어쩌면 ‘자궁건강 적신호’.'

In [16]:
content

'폐경이란 여성호르몬 감소로 인해 임신이 어려워지는 상태를 말한다. 일반적으로 40세 이상이 되면 자궁수축호르몬인 에스트로겐이 급격히 떨어지면서 생리를 하지 않을 때와 같은 양으로 많아지고, 그 사이 호르몬 수치가 정상수준으로 돌아온다. 폐경무렵이면 임신에 더 좋다는 속설이 있다. 그러나 폐경무렵이면 호르몬이 감소해 자궁건강에 좋지 않다. 특히 폐경무렵이면 정상체중에 살더라도 출산 후 여성호르몬인 에스트로겐 수치가 높아지기는커녕 오히려 수치가 상승한다. 폐경무렵부터 자궁건강을 지키려면 월경이 시작되더라도 생리예정일보다 하루 빨리 규칙적으로 병원을 방문해 자궁검사를 받는 것이 좋다. 여성미한의원 조선화 원장은 “폐경무렵은 자궁근육층을 형성하는 에스트로겐이나 프로게스테론 같은 여성호르몬을 생성하는 에스트로겐 수치가 가장 높은 시기”라고 설명했다. 자궁에 혹이 생겨 생리를 하지 않는다면 괜찮지만, 혹이 생리기 시작한 경우에는 자궁이 점점 커지고 자궁근육층이 두터워지면서 통증이 심해질 수 있다. 또 에스트로겐의 경우 자궁과 난소의 평활근을 강화시키고 여성 호르몬의 생성을 감소시키며 자궁내막증식작용, 자궁근종, 자궁내막염 등 다양한 질환을 발생시키기도 한다. 특히 에스트로겐 분비가 급격히 증가한 폐경무렵의 경우 호르몬 불균형으로 인해 자궁내막증식작용이 발생해 자궁출혈이나 자궁내막염 등이 나타날 수 있으며 자궁염, 자궁근종 등 자궁질환을 악화시킬 수 있다. 또 출산 이후 생리가 끊기는 경우도 많다. 호르몬 불균형으로 자궁에 만성적인 염증이 발생함으로써 자궁근종, 자궁내막증 등의 질환이 발생, 생리가 잘 이루어지지 않는 것이다. 따라서 평소 임신가능여부를 꼭 확인해야 하며 무리한 다이어트나 지나친 음주, 흡연 등 여러 가지 생활습관을 개선해야한다. 생리가 불규칙해지면 ‘태아형성장애’라 하여 자궁건강에 문제가 생겨난다. '